<a href="https://colab.research.google.com/github/badbloody/diploma2023/blob/main/fid_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import tensorflow as tf
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
import os
from PIL import Image

In [3]:
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)

        # store
        images_list.append(new_image)
    return asarray(images_list)

In [6]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2, batch_size=2):
    act1_list = []
    act2_list = []
    for i in range(0, len(images1), batch_size):
        batch_images1 = images1[i:i+batch_size]
        batch_images2 = images2[i:i+batch_size]

        act1_batch = model.predict(batch_images1)
        act2_batch = model.predict(batch_images2)

        act1_list.append(act1_batch)
        act2_list.append(act2_batch)

    act1 = np.concatenate(act1_list, axis=0)
    act2 = np.concatenate(act2_list, axis=0)

    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)

    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))

    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real

    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [7]:
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

# Load images from local folders
content_images_dir = "/content/gdrive/MyDrive/validation_images_10"
transformed_images_dir = "/content/gdrive/MyDrive/validation_transferred_10"
transformed_images_dir_mine = "/content/gdrive/MyDrive/validation_transferred_10_mine"

content_images = []
transformed_images = []
transformed_images_mine = []

for image_file in os.listdir(content_images_dir):
    image_path = os.path.join(content_images_dir, image_file)
    image = Image.open(image_path).convert("RGB")
    image = np.array(image)
    content_images.append(image)

    transformed_image_path = os.path.join(transformed_images_dir, image_file)
    transformed_image = Image.open(transformed_image_path).convert("RGB")
    transformed_image = np.array(transformed_image)
    transformed_images.append(transformed_image)

    transformed_image_path_mine = os.path.join(transformed_images_dir_mine, image_file)
    transformed_image_2 = Image.open(transformed_image_path_mine).convert("RGB")
    transformed_image_2 = np.array(transformed_image_2)
    transformed_images_mine.append(transformed_image_2)

content_images = np.array(content_images)
transformed_images = np.array(transformed_images)
transformed_images_mine = np.array(transformed_images_mine)

# convert integer to floating point values
images1 = content_images.astype('float32')
images2 = transformed_images.astype('float32')
images3 = transformed_images_mine.astype('float32')

# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
images3 = scale_images(images3, (299,299,3))

# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
images3 = preprocess_input(images3)

batch_size = 2
fid = calculate_fid(model, images1, images2, batch_size)
print('FID of original method: %.3f' % fid)

# Clear TensorFlow session to release GPU memory
tf.keras.backend.clear_session()

fid = calculate_fid(model, images1, images3, batch_size)
print('FID of my method: %.3f' % fid)

1/1 [==============================] - 0s 27ms/step
FID of original method: 459.828
1/1 [==============================] - 0s 28ms/step
FID of my method: 364.391
